# Les index

* Auteurs/trices : Adrien VIGUE, Etienne BARON, Camille MONOT

Ce chapitre traite de :
* Index + requêtes textuelles + requêtes géographiques


## À quoi servent-ils ?

Les index prennent en charge l'exécution efficace des requêtes dans MongoDB. Sans index, MongoDB doit effectuer une analyse de collection , c'est-à-dire analyser chaque document d'une collection, pour sélectionner les documents qui correspondent à l'instruction de requête. Si un index approprié existe pour une requête, MongoDB peut utiliser l'index pour limiter le nombre de documents qu'il doit inspecter. Les index de MongoDB sont similaires aux index d'autres systèmes de base de données. MongoDB définit les index au niveau de la collection et prend en charge les index sur n'importe quel champ ou sous-champ des documents dans une collection MongoDB.

En bref, les index sont un moyen de trouver rapidement et efficacement du contenu dans une base de données.

## Quand les utiliser ?

On met en place des index à chaque fois qu'on s'attend à avoir beaucoup de requêtes sur une clé (resp. un ensemble de clés). 
Par exemple, on souhaite récupérer les notes d'un étudiant à l'aide de son numéro étudiant. Il faudra alors mettre un index sur la clé "numéro étudiant" afin des réaliser des requetes efficaces pour retrouver toutes les informations de l'étudiant en fonction de son numéro.

```{admonition} ⚠️ Attention
:class: tip
On ne peut pas toujours utiliser des index. En effet, les index rendent la mise à jour de la base fastidieuse : à chaque fois que vous
rajoutez de nouvelles données, vous devez redéfinir vos index. Il n'est donc pas judicieux d'utiliser des index sur une base où vous vous
attendez à devoir faire des mises à jour régulières.
```

## Syntaxe adaptée

* Création d'un index

```javascript
db.collec.createIndex({"cle":1})
```
On crée ici un index dans la collection 'collec' de la base de données courante 'db'. On donne ensuite le nom du champs sur lequel on va ensuite créer la clé. Le ":1" signifie que l'index va trier les données dans l'ordre croissant.

_Exemple :_

In [1]:
use food

switched to db food

In [2]:
db.NYfood.createIndex({"borough" : 1})

{
	"numIndexesBefore" : 5,
	"numIndexesAfter" : 5,
	"note" : "all indexes already exist",
	"ok" : 1
}

Ici on crée un index "quartier" dans la collection NYfood de la base food. Le quartier étant une information importante du restaurant, il est judicieux de créer un index pour toutes les requêtes ultérieures. 

```{admonition} ✍ À noter
Si on avait mis une valeur négative à la place du 1, les données auraient été triées dans l'ordre décroissant. 
Le tri n'a cependant pas d'importance sur l'efficacité de la requête.
```
* Récupération d'index

Lorsque l'on prend en main une nouvelle base mongoDB, il est judicieux de se renseigner sur les index déja créés, afin de construire les requêtes les plus optimisées possible. Pour cela, on utilise getIndexes :

```javascript
db.collec.getIndexes()
```

La syntaxe est très simple et limpide, et donc si l'on applique cette dernière à notre exemple :

In [3]:
db.NYfood.getIndexes()

[
	{
		"v" : 2,
		"key" : {
			"_id" : 1
		},
		"name" : "_id_"
	},
	{
		"v" : 2,
		"key" : {
			"address.loc" : "2dsphere"
		},
		"name" : "address.loc_2dsphere",
		"2dsphereIndexVersion" : 3
	},
	{
		"v" : 2,
		"key" : {
			"_fts" : "text",
			"_ftsx" : 1
		},
		"name" : "$**_text",
		"weights" : {
			"$**" : 1
		},
		"default_language" : "english",
		"language_override" : "language",
		"textIndexVersion" : 3
	},
	{
		"v" : 2,
		"key" : {
			"borough" : 1
		},
		"name" : "borough_1"
	},
	{
		"v" : 2,
		"key" : {
			"cuisine" : 1
		},
		"name" : "cuisine_1"
	}
]

* Opérateurs bénéficiant de l’index

Construire une requête mongoDB utilisant des index ne difèrent pas d'une requête n'en utilisant pas, toutefois, certains opérateurs logiques bénéficient tout particlulièrement de la présence d'un ou plusieurs index. Il est donc pertinent de construire des index si vous pensez utilisez ces opérateurs.

_Exemple 1 : Opérateur égal (:, $eq)_

In [4]:
db.NYfood.find({"cuisine": "Chinese", "borough": "Brooklyn"})

{
	"_id" : ObjectId("6067925c0cd9232287d9c769"),
	"address" : {
		"building" : "1269",
		"loc" : {
			"type" : "Point",
			"coordinates" : [
				-73.871194,
				40.6730975
			]
		},
		"street" : "Sutter Avenue",
		"zipcode" : "11208"
	},
	"borough" : "Brooklyn",
	"cuisine" : "Chinese",
	"grades" : [
		{
			"date" : ISODate("2014-09-16T00:00:00Z"),
			"grade" : "B",
			"score" : 21
		},
		{
			"date" : ISODate("2013-08-28T00:00:00Z"),
			"grade" : "A",
			"score" : 7
		},
		{
			"date" : ISODate("2013-04-02T00:00:00Z"),
			"grade" : "C",
			"score" : 56
		},
		{
			"date" : ISODate("2012-08-15T00:00:00Z"),
			"grade" : "B",
			"score" : 27
		},
		{
			"date" : ISODate("2012-03-28T00:00:00Z"),
			"grade" : "B",
			"score" : 27
		}
	],
	"name" : "May May Kitchen",
	"restaurant_id" : "40358429"
}
{
	"_id" : ObjectId("6067925c0cd9232287d9c78b"),
	"address" : {
		"building" : "976",
		"loc" : {
			"type" : "Point",
			"coordinates" : [
				-73.92701509999999,
				40.6620192
			]
		},
		"stre

On récupère les restaurants proposant de la cuisine chinoise dans le quartier de Brooklyn.

_Exemple 2 : Opérateur infériorité/supériorité ($lt, $lte, $gt, $gte)_

In [5]:
db.users.find({"age": 20,"name": {$gte: "user100000", $lte:"user100000"}})

On récupère les utilisateurs de 20 ans et dont l'id est compris entre 10 000 et 100 000.

## Index composés

## Index textuels
* Création d'un index
* Requêtes avancées
## Index géo-spatiaux

Les index geo-spatiaux ...
* Création d'un index

```javascript
db.coll.createIndex({"att" : "2dsphere"})
```

* Requêtes avancées
```javascript
var ref = {"type": "Point", "coordinates": [longitude, latitude]}
db.nomDeLaCollection.find({"clé": {$near : {$geometry : ref}}})
```
_Exemple :_

In [6]:
var CrownHeights= {"type": "Point", "coordinates": [-73.923, 40.676]}
db.NYfood.find({"address.loc" : {$near: {$geometry: CrownHeights}}})

uncaught exception: SyntaxError: unexpected token: identifier :
@(shell):1:70

```javascript
var ref = {"type": "Polygon", "coordinates": [[[long1, lat1],
                                                 [long2, lat2],
                                                 [long3, lat3],
                                                 [long4, lat4],
                                                 [long1, lat1]]]}
db.nomDeLaCollection.find({"clé": {$within : {$geometry : ref}}})                                                 
```
_Exemple :_

In [7]:
eastVillage= {"type" : "Polygon", "coordinates" : [[[-73.9917900, 40.7264100],
                                                    [-73.9917900, 40.7321400],
                                                    [-73.9829300, 40.7321400],
                                                    [-73.9829300, 40.7264100],
                                                    [-73.9917900, 40.7264100]]]}
db.NYfood.find({"address.loc": {$within : {$geometry : eastVillage}}})

uncaught exception: SyntaxError: unexpected token: identifier :
@(shell):1:189